## Pruning the network

In [1]:
!pip install tensorflow-model-optimization

In [2]:
!pip show tensorflow-model-optimization

Name: tensorflow-model-optimization
Version: 0.8.0
Summary: A suite of tools that users, both novice and advanced can use to optimize machine learning models for deployment and execution.
Home-page: https://github.com/tensorflow/model-optimization
Author: Google LLC
Author-email: no-reply@google.com
License: Apache 2.0
Location: C:\Users\yangw\anaconda3\Lib\site-packages
Requires: absl-py, dm-tree, numpy, six
Required-by: 


In [3]:
!pip install tf-keras 

In [4]:
!!pip --version

['pip 24.0 from C:\\Users\\yangw\\anaconda3\\Lib\\site-packages\\pip (python 3.12)',
 '']

In [5]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_model_optimization as tfmot

In [6]:
import tempfile
import os

import tensorflow as tf
import numpy as np

from tensorflow import keras

In [7]:
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0

model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=4, validation_split=0.1)




Epoch 1/4


1688/1688 [==============================] - 7s 3ms/step - loss: 0.2741 - accuracy: 0.9232 - val_loss: 0.1072 - val_accuracy: 0.9722
Epoch 2/4
1688/1688 [==============================] - 5s 3ms/step - loss: 0.1036 - accuracy: 0.9699 - val_loss: 0.0802 - val_accuracy: 0.9800
Epoch 3/4
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0790 - accuracy: 0.9768 - val_loss: 0.0702 - val_accuracy: 0.9805
Epoch 4/4
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0680 - accuracy: 0.9793 - val_loss: 0.0665 - val_accuracy: 0.9820


In [8]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

Baseline test accuracy: 0.9782999753952026


In [9]:
!pip install tensorflow-model-optimization

In [10]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_model_optimization as tfmot

initializer = keras.regularizers.l1(0.01)
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(400, activation="relu", kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(100, activation="relu", kernel_regularizer=initializer),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])

opt = keras.optimizers.Adam(learning_rate=0.01, clipnorm=1.0)
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt)

# Apply pruning
model_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(model)

# Recompile the model after pruning
model_for_pruning.compile(optimizer='adam',
                          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                          metrics=['accuracy'])

model_for_pruning.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_flatte  (None, 784)               1         
 n_1 (PruneLowMagnitude)                                         
                                                                 
 prune_low_magnitude_dense_  (None, 300)               470702    
 1 (PruneLowMagnitude)                                           
                                                                 
 prune_low_magnitude_dense_  (None, 400)               240402    
 2 (PruneLowMagnitude)                                           
                                                                 
 prune_low_magnitude_dense_  (None, 100)               80102     
 3 (PruneLowMagnitude)                                           
                                                                 
 prune_low_magnitude_dropou  (None, 100)              

In [11]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(train_images, train_labels,
                  batch_size=128, epochs=2, validation_split=0.1, callbacks=callbacks)

Epoch 1/2


C:\Users\yangw\anaconda3\Lib\site-packages\tf_keras\src\backend.py:5729: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


422/422 [==============================] - 9s 8ms/step - loss: 3.4859 - accuracy: 0.8490 - val_loss: 0.6424 - val_accuracy: 0.9348
Epoch 2/2
422/422 [==============================] - 3s 7ms/step - loss: 0.6462 - accuracy: 0.9098 - val_loss: 0.4817 - val_accuracy: 0.9478


In [12]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy) 
print('Pruned test accuracy:', model_for_pruning_accuracy)

Baseline test accuracy: 0.9782999753952026
Pruned test accuracy: 0.9343000054359436
